In [48]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_21_1_250.h5",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

# Creating env objects

In [30]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-M

In [31]:
type(env._ref_traj.quaternion)

numpy.ndarray

In [32]:
env.body_idxs

Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17], dtype=int32)

In [47]:
# env._ref_traj.body_positions[:,env.body_idxs]

In [34]:
env._ref_traj.angular_velocity.shape

(3, 250)

In [35]:
# def slice_arrays(start_index, end_index):
#     def slicer(x):
#         if isinstance(x, jp.ndarray):
#             return x[..., start_index:end_index]
#         return x
#     return slicer

# sliced_tree = jax.tree_map(slice_arrays(0, 5), env._ref_traj)
# sliced_tree.body_positions.shape

In [36]:
start_index = 0
end_index = 5
sliced_clip = env._ref_traj.slice_clip(start_index, end_index)
sliced_clip.center_of_mass.shape

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/environments.py:41: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(slicer, self)


(3, 5)

# Creating state objects & reset_fn, get_obs_fn
`reset_fn` and `get_obs_fn` is called when initilizing the environment and creating the state object

In [37]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx

In [38]:
state.info['start_frame']

Array(56, dtype=int32)

In [39]:
env._ref_traj.velocity[:, state.info['start_frame']]

array([ 0.40821564, -0.20567179, -0.11344627])

In [40]:
state.pipeline_state.xpos.shape

(66, 3)

In [41]:
env._ref_traj.body_positions.shape

(54, 250)

In [42]:
state.obs.shape

(1735,)

end effector position

In [43]:
state.pipeline_state.xpos[jp.array(env._end_eff_idx)]

Array([[-0.00912952,  0.4411053 ,  0.01141115],
       [-0.03021844,  0.414083  ,  0.01621009],
       [ 0.0780039 ,  0.40200663,  0.01433735],
       [ 0.08267315,  0.36600125,  0.02249428]], dtype=float32)

In [44]:
env._ref_traj.end_effectors.shape

(12, 250)

# Creating step_fn, reward_fn, & termination_fn
`step_fn`, which calls `reward_fn` & `termination_fn` is only used when actually making action, not when setting up environment and state object

In [45]:
# rollout = [state.pipeline_state]
# ctrl = jax.numpy.array(np.random.uniform(-1,1, env.sys.nu))
# state = env.step(state, ctrl)
# rollout.append(state.pipeline_state)

In [46]:
# media.show_video(env.render(rollout,camera='close_profile'), fps=1.0 / env.dt)